In [1]:
import pandas as pd

df_r = pd.read_excel('df_x.xlsx')

In [2]:
df_r.head()

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
0,106.513158,86.973684,80.460526,67.434211,46.592105,67.434211,53.105263,85.671053,55.710526,38.776316,...,57.013158,53.105263,37.473684,60.921053,66.131579,51.802632,54.407895,54.407895,42.684211,41.381579
1,86.973684,106.513158,85.671053,84.368421,58.315789,79.157895,50.500000,83.065789,49.197368,40.078947,...,43.986842,42.684211,28.355263,55.710526,66.131579,51.802632,55.710526,62.223684,46.592105,47.894737
2,80.460526,85.671053,106.513158,81.763158,75.250000,75.250000,63.526316,70.039474,40.078947,50.500000,...,60.921053,60.921053,46.592105,75.250000,84.368421,71.342105,75.250000,71.342105,67.434211,70.039474
3,67.434211,84.368421,81.763158,106.513158,75.250000,97.394737,71.342105,84.368421,58.315789,60.921053,...,38.776316,40.078947,24.447368,57.013158,63.526316,47.894737,62.223684,76.552632,68.736842,70.039474
4,46.592105,58.315789,75.250000,75.250000,106.513158,80.460526,70.039474,64.828947,54.407895,59.618421,...,40.078947,38.776316,34.868421,50.500000,63.526316,47.894737,63.526316,85.671053,90.881579,88.276316


In [3]:
import gurobipy as gp
from gurobipy import GRB

# 데이터 정의
C = 2300000000000
S = 18900
T = [29910000, 15420000, 16244000, 9325000, 8723000, 6040000, 3660000, 8892000, 7109000, 4733000, 
     8724000, 4447000, 5498000, 6916000, 9440000, 7406000, 9123000, 6124000, 11410000, 19860000, 
     9176000, 21020000, 20000000, 32720000, 8815000]
H = [156997, 146399, 230745, 274801, 349120, 388156, 403956, 427771, 294677, 306342, 
     503110, 444865, 304655, 397122, 433382, 631419, 392651, 283711, 372231, 438208, 
     559566, 422258, 536385, 729517, 429936]
I = [5220000, 4950000, 6980000, 5250000, 4080000, 3790000, 3280000, 4220000, 3100000, 3390000, 
     4010000, 3690000, 4460000, 5120000, 4940000, 3950000, 3740000, 3250000, 4750000, 4420000, 
     3370000, 8050000, 8110000, 5570000, 4420000]

# Rij 행렬 (예시 데이터, 실제 데이터로 대체 필요)
import numpy as np
# R = np.random.rand(25, 25) # 임의의 예시 데이터
R = df_r.to_numpy()

# 모델 초기화
model = gp.Model("BuildingOptimization")

# 변수 정의
X = model.addVars(25, vtype=GRB.BINARY, name="X")
Y = model.addVars(25, 25, vtype=GRB.BINARY, name="Y")

# 제약식 5를 통해 A 계산
sum_Ri19_Hi_Ii = sum(R[i, 18] * H[i] * I[i] for i in range(25))
A = (7100000000000 + C + T[18]) / sum_Ri19_Hi_Ii
print(f'A : {A}')
# 목적함수 정의
model.setObjective(
    -gp.quicksum((C + S * T[j]) * X[j] for j in range(25)) + 
    A * gp.quicksum(R[i, j] * H[i] * I[i] * Y[i, j] for i in range(25) for j in range(25)),
    GRB.MAXIMIZE
)

# 제약식 1: 건물은 최대 2개까지 가능하다.
model.addConstr(gp.quicksum(X[j] for j in range(25)) <= 2)

# 제약식 2: j 지역의 건물을 세워야만 i지역으로부터 연결이 가능하다.
for i in range(25):
    for j in range(25):
        model.addConstr(Y[i, j] <= X[j])

# 제약식 3: 각 i는 하나의 지역 j와만 연결된다.
for i in range(25):
    model.addConstr(gp.quicksum(Y[i, j] for j in range(25)) == 1)

# 제약식 4: 지역 19번은 무조건 연결된다.
model.addConstr(X[18] == 1)

# 최적화
model.optimize()

# 결과 출력
if model.status == GRB.OPTIMAL:
    selected_buildings = [j for j in range(25) if X[j].X > 0.5]
    connections = [(i, j) for i in range(25) for j in range(25) if Y[i, j].X > 0.5]
    print("Selected buildings:", selected_buildings)
    print("Connections:", connections)
else:
    print("Optimal solution was not found.")


Restricted license - for non-production use only - expires 2025-11-24
A : 0.0033851344710018167
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 652 rows, 650 columns and 1901 nonzeros
Model fingerprint: 0xee6eac56
Variable types: 0 continuous, 650 integer (650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+09, 3e+12]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 380 rows and 356 columns
Presolve time: 0.01s
Presolved: 272 rows, 294 columns, 794 nonzeros
Variable types: 0 continuous, 294 integer (294 binary)
Found heuristic solution: objective 6.619104e+12
Found heuristic solution: object

In [1]:
connection = [(0, 18), (1, 4), (2, 18), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (11, 18), (12, 18), (13, 18), (14, 18), (15, 18), (16, 18), (17, 18), (18, 18), (19, 18), (20, 18), (21, 4), (22, 4), (23, 4), (24, 4)]



In [2]:
gu = ["종로구",
"중구",
"용산구",
"성동구",
"광진구",
"동대문구",
"중랑구",
"성북구",
"강북구",
"도봉구",
"노원구",
"은평구",
"서대문구",
"마포구",
"양천구",
"강서구",
"구로구",
"금천구",
"영등포구",
"동작구",
"관악구",
"서초구",
"강남구",
"송파구",
"강동구",]

In [3]:
gwang = []
young = []
for i in connection:
    if i[-1] == 4:
        gwang.append(gu[i[0]])
    else:
        young.append(gu[i[0]])
        
print(f'광진구 : {gwang}')
print(f'영등포구 : {young}')

광진구 : ['중구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '서초구', '강남구', '송파구', '강동구']
영등포구 : ['종로구', '용산구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구']
